# Python 零售銀行機器學習

> 機器學習模組：Scikit-Learn

[郭耀仁](https://hahow.in/@tonykuoyj?tr=tonykuoyj) | yaojenkuo@ntu.edu.tw

## 更多面對預測任務的模型

## 面對預測任務的模型

- 正規方程。
- 羅吉斯回歸。
- k 最近鄰。
- 高斯單純貝氏分類器。
- 決策樹。

## 什麼是 k 最近鄰

- k 最近鄰（k-Nearest Neighbors, KNN）是一種基於資料之間的相似度來決定是否為同一類別的演算方法。
- 「歐幾里德距離 Euclidean distance」是最常用來量測資料相似度的指標，歐幾里德距離以白話文敘述其實就是直線距離。

\begin{align}
d(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}
\end{align}

## 什麼是 k 最近鄰（續）

- 更為泛用的距離量測是「明可夫斯基距離 Minkowski distance」。
- 當式子中的 $p=1$ 時就是曼哈頓距離、$p=2$ 時就是歐幾里德距離。

\begin{align}
d(x, y) = \left( \sum_{i=1}^{n} \mid x_i - y_i \mid ^p \right)^{\frac{1}{p}}
\end{align}

## 什麼是 k 最近鄰（續）

- k 最近鄰會根據預測資料點周遭的 k 個最相似訓練資料點決定分類結果，k 可以由使用者自行決定。
- 在二元分類的範疇下，k 會選擇一個奇數使得分類結果直接被決定。
- k 最近鄰模型的訓練與預測在同時間發生，也就是訓練在輸入預測資料時才發生，因此屬於 Lazy learning 的機器學習方法。

## 自行定義 k 最近鄰預測器類別

```python
class kNN:
    def __init__(self, n_neighbors=5):
        self._n_neighbors = n_neighbors
    def minkowski_distances(self, X_train, X_test_i, p=2):
        distances = (np.sum((np.abs(X_test_i - X_train))**p, axis=1))**(1/p)
        return distances
    def fit(self, X_train, y_train):
        self._X_train = X_train
        self._y_train = y_train
    def predict(self, X_test, p=2):
        nrows = X_test.shape[0]
        y_test = []
        for i in range(nrows):
            X_test_i = X_test[i, :]
            distances = self.minkowski_distances(self._X_train, X_test_i)
            y_train_reshape = self._y_train.reshape(-1, 1)
            distances_reshape = distances.reshape(-1, 1)
            distances_y_train = np.concatenate((distances_reshape, y_train_reshape), axis=1)
            distances_y_train_argsort = distances_y_train[distances_y_train[:, 0].argsort()]
            k_nearest_labels = distances_y_train_argsort[:self._n_neighbors, 1].astype(int)
            labels, counts = np.unique(k_nearest_labels, return_counts=True)
            argmax_counts = np.argmax(counts)
            y_test_i = labels[argmax_counts]
            y_test.append(y_test_i)
        return np.array(y_test)
```

## 高斯單純貝氏分類器

## 什麼是高斯單純貝氏分類器

- 高斯單純貝氏分類器是基於貝氏定理的分類模型。
- 貝氏定理是在先驗機率（Prior probability）的基礎上，納入新事件的資訊來更新先驗機率，得到後驗機率（Posterior probability）的統計分法。

\begin{align}
P(y|x_i) = \frac{P(x_i|y) \times P(y)}{P(x_i)} \\
\text{posterior} = \frac{\text{likelihood} \times \text{prior}}{\text{evidence}}
\end{align}

## 什麼是高斯單純貝氏分類器（續）

- 高斯單純貝氏分類器中的「單純」指的是在計算分類機率時，會假設資料特徵不依賴類別，兩者彼此獨立。
- 因此實際在計算後驗機率的時候，只需要關注分子的部分。

\begin{align}
P(y|x_i) \propto P(x_i|y) P(y) \\
y^* = argmax_y \, P(x_i|y) P(y) \\
y^* = argmax_y \, P(y) \prod P(x_i|y)
\end{align}

## 什麼是高斯單純貝氏分類器（續）

當特徵為連續型變數時，可以藉由假設變數為常態分配的情況下，以樣本資料的平均數及標準差來計算機率（Likelihood），也就是 $P(x_i|y)$ 能以高斯機率密度函數計算。

\begin{align}
P(x_i | y) = \frac{1}{\sqrt{2 \pi \sigma_y^2}} exp \left( -\frac{(x_i - \mu_y)^2}{2 \sigma_y^2} \right)
\end{align}

## 什麼是高斯單純貝氏分類器（續）

當特徵數量很多的時候，$\prod P(x_i|y)$ 相乘所算出的機率值會非常小，造成後驗機率趨近於 0，這時可以透過取對數函數來避免。

\begin{align}
y^* = argmax_y \, log \left(P(y) \right) + \sum log \left( P(x_i|y) \right)
\end{align}

## 自行定義高斯單純貝氏分類器的類別

```python
class GaussianNaiveBayes:
    def get_prior_proba(self, y_train):
        values, counts = np.unique(y_train, return_counts=True)
        prior_proba = dict()
        for value, count in zip(values, counts):
            prior_proba[value] = count / y_train.size
        return prior_proba
    def get_mean_var(self, X_train, y_train):
        n_features = X_train.shape[1]
        X_y_train = np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1)
        unique_labels = np.unique(y_train)
        X_mean_var = dict()
        for unique_label in unique_labels:
            X_y_train_label = X_y_train[X_y_train[:, -1] == unique_label]
            label_mean = np.mean(X_y_train_label[:, 0:-1], axis=0)
            label_var = np.var(X_y_train_label[:, 0:-1], axis=0)
            X_mean_var[unique_label] = {
                "mean": label_mean,
                "var": label_var
            }
        return X_mean_var
```

```python
    def fit(self, X_train, y_train):
        prior_proba = self.get_prior_proba(y_train)
        mean_var = self.get_mean_var(X_train, y_train)
        self._prior_proba = prior_proba
        self._mean_var = mean_var
    def get_likelihood(self, X_test_i, var, mean):
        multiplier = 1 / np.sqrt(2 * np.pi * var)
        numerator = (X_test_i - mean)**2
        denominator = 2*var
        exponenet = np.exp(- numerator / denominator)
        likelihood = multiplier * exponenet
        return likelihood
```

```python
    def predict(self, X_test):
        n_rows = X_test.shape[0]
        labels = self._prior_proba.keys()
        y_preds = []
        for i in range(n_rows):
            X_test_i = X_test[i, :]
            log_posterior_probas = []
            for label in labels:
                label_var = self._mean_var[label]["var"]
                label_mean = self._mean_var[label]["mean"]
                likelihood = self.get_likelihood(X_test_i, label_var, label_mean)
                log_likelihood = np.log(likelihood)
                sum_log_likelihood = np.sum(log_likelihood)
                log_prior_proba = np.log(self._prior_proba[label])
                log_posterior_proba = log_prior_proba + sum_log_likelihood
                log_posterior_probas.append(log_posterior_proba)
            log_posterior_probas = np.array(log_posterior_probas)
            y_pred = np.argmax(log_posterior_probas)
            y_preds.append(y_pred)
        return np.array(y_preds)
```

## 決策樹

## 什麼是決策樹

- 決策樹（Decision tree）是一種利用外型像樹一樣的圖形決策模型，具有快速、可解釋性高的優點。
- 決策樹需要從資料中尋找合適的「特徵」與「切點」來進行樹的分支，多次分支後企圖讓資料有高差異性的分類。
- 例如決策是否要跑一場馬拉松：
    - 氣溫是否低於 15 度？
        - 否，不要跑。
        - 是。
            - 濕度是否低於 60%？
                - 否，不要跑。
                - 是，要跑。

## 什麼是決策樹（續）

建立一個決策樹模型必須要考量三個要素：

1. 要使用資料中的哪個變數作為特徵。
2. 要如何決定特徵的切點。
3. 何時要停止分支。

## 什麼是決策樹（續）

- 使用演算法計算資訊增益（Information Gain）、熵（Entropy）、資訊增益率（Information Gain Ratio）或吉尼不純度（Gini Impurity）來決定前述三要素。
- 因此建構決策樹的演算法可再分為：
    - ID3(Iterative Dichotomiser 3)
    - C4.5
    - C5.0
    - CART

## 自行定義決策樹的類別

```python
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, information_gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.information_gain = information_gain
        self.value = value
```

```python
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
    def get_entropy(self, x):
        epsilon = 1e-6
        values, counts = np.unique(x, return_counts=True)
        p = counts / counts.sum()
        entropy = np.sum(-p * np.log2(p + epsilon)) # to avoid log2(0)
        return(entropy)
    def get_information_gain(self, parent_node, left_child_node, right_child_node):
        parent_entropy = self.get_entropy(parent_node)
        p_left = left_child_node.size / parent_node.size
        p_right = right_child_node.size / parent_node.size
        child_entropy = p_left*self.get_entropy(left_child_node) + p_right*self.get_entropy(right_child_node)
        information_gain = parent_entropy - child_entropy
        return information_gain
```

```python
    def get_best_split(self, X_train, y_train):
        best_split = dict()
        best_information_gain = -1
        n_rows, n_cols = X_train.shape
        for col_idx in range(n_cols):
            X_col_i = X_train[:, col_idx]
            uniques = np.unique(X_col_i)
            for threshold in uniques:
                X_train_y_train = np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1)
                Xy_left = X_train_y_train[X_col_i <= threshold]
                Xy_right = X_train_y_train[X_col_i > threshold]
                if Xy_left.shape[0] > 0 and Xy_right.shape[0] > 0:
                    y_parent = X_train_y_train[:, -1]
                    y_left = Xy_left[:, -1]
                    y_right = Xy_right[:, -1]
                    information_gain = self.get_information_gain(y_parent, y_left, y_right)
                    if information_gain > best_information_gain:
                        best_split = {
                            "column_index": col_idx,
                            "threshold": threshold,
                            "Xy_left": Xy_left,
                            "Xy_right": Xy_right,
                            "information_gain": information_gain
                        }
                        best_information_gain = information_gain
        return best_split
```

```python
    def build(self, X_train, y_train, depth=0):
        n_rows, n_cols = X_train.shape
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            best_split = self.get_best_split(X_train, y_train)
            if best_split["information_gain"] > 0:
                X_left = best_split["Xy_left"][:, :-1]
                y_left = best_split["Xy_left"][:, -1]
                left_node = self.build(X_left,
                                       y_left,
                                       depth = depth + 1)
                X_right = best_split["Xy_right"][:, :-1]
                y_right = best_split["Xy_right"][:, -1]
                right_node = self.build(X_right,
                                        y_right,
                                        depth = depth + 1)
                return Node(feature = best_split["column_index"],
                            threshold = best_split["threshold"],
                            left = left_node,
                            right = right_node,
                            information_gain = best_split["information_gain"])
        values, counts = np.unique(y_train, return_counts=True)
        argmax_counts = np.argmax(counts)
        most_common = values[argmax_counts]
        return Node(value=most_common)
    def fit(self, X_train, y_train):
        self.root = self.build(X_train, y_train)
```

```python
    def predict_X_test_i(self, X_test_i, tree):
        if tree.value != None:
            return tree.value
        col_idx = tree.feature
        feature_value = X_test_i[col_idx]
        if feature_value <= tree.threshold:
            return self.predict_X_test_i(X_test_i, tree.left)
        if feature_value > tree.threshold:
            return self.predict_X_test_i(X_test_i, tree.right)
    def predict(self, X_test):
        n_rows = X_test.shape[0]
        y_test = []
        for i in range(n_rows):
            X_test_i = X_test[i, :]
            y_test_i = self.predict_X_test_i(X_test_i, self.root)
            y_test.append(y_test_i)
        return np.array(y_test)
```

## 這還只是我們列出來的很小一部分

- 正規方程。
- 羅吉斯回歸。
- k 最近鄰。
- 高斯單純貝氏分類器。
- 決策樹。
- 隨機森林。
- 梯度遞增(XGBoost)。
- ...etc.

## scikit-learn to the rescue!

## 關於 Scikit-Learn

## 什麼是 Scikit-Learn

> Scikit-learn 是 Python 機器學習的第三方模組，透過它可以進行監督式以及非監督式學習，提供了模型訓練、資料預處理、模型選擇以及模型評估等功能。

來源：<https://scikit-learn.org>

## （沒什麼用的冷知識）Scikit-Learn 是最受歡迎的 SciKit(SciPy Toolkit)

- Scikit-Learn 與 Scikit-Image 是兩個最受歡迎、維護最良善的 Scikits
- 還有眾多其他的 Scikits

來源：<https://projects.scipy.org/scikits.html>

## 根據說明文件的範例載入

多數時候我們使用 Scikit-Learn 中的特定類別或函數，因此以 `from sklearn import FUNCTION/CLASS` 載入特定類別或函數，而非 `import sklearn`

來源：<https://scikit-learn.org/stable/getting_started.html>

In [1]:
import sklearn # use `from sklearn import FUNCTION/CLASS` instead

## 如果環境中沒有安裝 Scikit-Learn，載入時會遭遇 `ModuleNotFoundError`

```
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'sklearn'
```

## 如果遭遇 `ModuleNotFoundError` 可以在終端機使用 `pip install scikit-learn` 或者 `conda install scikit-learn` 指令安裝

若要指定模組版本可以加上 `==MAJOR.MINOR.PATCH` 課程使用的模組版本為 1.0

```bash
pip install scikit-learn==1.0
```
或者

```bash
conda install scikit-learn==1.0
```

## 可以透過兩個屬性檢查版本號與安裝路徑

- `__version__` 屬性檢查版本號。
- `__file__` 屬性檢查安裝路徑。

In [2]:
print(sklearn.__version__)
print(sklearn.__file__)

1.3.0
/Users/kuoyaojen/miniconda3/envs/datascience/lib/python3.11/site-packages/sklearn/__init__.py


## 為什麼選擇 Scikit-Learn

- 簡潔、一致且設計良善的應用程式介面設計，只要理解基礎用法和語法，就能延伸切換到其他的演算法或模型。
- 文件撰寫完整而豐富。
- 維護良善。

## Scikit-Learn 應用程式介面設計原則

1. 一致性。
2. 可檢查性。
3. 不擴增新類別。
4. 可組合性。
5. 合理預設參數。

## 使用轉換器預處理資料

## 轉換器與預測器是 Scikit-Learn 所創造最重要的兩種類別

1. **轉換器（Transformers）：用來預處理資料**。
2. 預測器（Predictors）：用來訓練模型、生成規則 $w$

## 使用 Scikit-Learn 轉換器的標準步驟

1. 準備欲轉換的特徵矩陣 $X$ 或目標陣列 $y$
2. 建立轉換器類別的物件。
3. 將欲轉換的特徵矩陣 $X$ 或目標陣列 $y$ 輸入 `transformer.fit_transform()`
4. 檢查轉換結果。

## 使用 Scikit-Learn 轉換器 `PolynomialFeatures`

生成一個指定次方數的特徵多項式矩陣。

In [1]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

csv_url = "https://raw.githubusercontent.com/datainpoint/classroom-python-for-finance-2025/refs/heads/main/ames/train.csv"
ames_train = pd.read_csv(csv_url) # import data
X = ames_train["OverallQual"].values.reshape(-1, 1)    # step 1
polynomial_features = PolynomialFeatures()             # step 2
X_transformed = polynomial_features.fit_transform(X)   # step 3
print(X_transformed[:5])                               # step 4

[[ 1.  7. 49.]
 [ 1.  6. 36.]
 [ 1.  7. 49.]
 [ 1.  7. 49.]
 [ 1.  8. 64.]]


## 使用 Scikit-Learn 轉換器 `StandardScaler`

生成一個經過 z-score 標準化的特徵矩陣。

\begin{equation}
z = \frac{x - \mu}{\sigma}
\end{equation}

In [2]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler()                     # step 2
X_transformed = standard_scaler.fit_transform(X)       # step 3
print(X_transformed[:5])                               # step 4

[[ 0.65147924]
 [-0.07183611]
 [ 0.65147924]
 [ 0.65147924]
 [ 1.3747946 ]]


## 兩種表示類別向量的形式

- 標籤編碼(LabelEncoder)。
- 讀熱編碼(OneHotEncoder)。

## 標籤編碼（Label encoder）

將類別變數的獨一值用 0 到 `n_classes - 1` 的整數表示，可以使用 Scikit-Learn 中的 `LabelEncoder` 轉換。

In [4]:
from sklearn.preprocessing import LabelEncoder

csv_url = "https://raw.githubusercontent.com/datainpoint/classroom-python-for-finance-2025/refs/heads/main/titanic/train.csv"
titanic_train = pd.read_csv(csv_url)
le = LabelEncoder()
gender = titanic_train["Sex"].values
gender_le = le.fit_transform(gender)
print(gender[:10])
print(gender_le[:10])

['male' 'female' 'female' 'female' 'male' 'male' 'male' 'male' 'female'
 'female']
[1 0 0 0 1 1 1 1 0 0]


## 讀熱編碼（OneHot encoder）

將類別變數的獨一值用 0 到 `n_classes - 1` 的整數表示，可以使用 Scikit-Learn 中的 `LabelEncoder` 轉換。

In [5]:
from sklearn.preprocessing import OneHotEncoder

oe = OneHotEncoder()
gender = titanic_train["Sex"].values
gender_oe = oe.fit_transform(gender.reshape(-1, 1)).toarray()
print(gender[:10])
print(gender_oe[:10, :])

['male' 'female' 'female' 'female' 'male' 'male' 'male' 'male' 'female'
 'female']
[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


## 使用預測器訓練及預測資料

## （複習）轉換器與預測器是 Scikit-Learn 所創造最重要的兩種類別

1. 轉換器（Transformers）：用來預處理資料。
2. **預測器（Predictors）：用來訓練模型、生成規則 $w$**

## 使用 Scikit-Learn 預測器的標準步驟

1. 準備欲訓練預測的特徵矩陣 $X$  與目標陣列 $y$
2. 切割訓練與驗證資料。
3. 建立預測器類別的物件。
4. 將訓練特徵矩陣 $X^{train}$ 與目標陣列 $y^{train}$ 輸入 `predictor.fit()`
5. 將驗證特徵矩陣 $X^{valid}$ 輸入 `predictor.predict()` 獲得 $\hat{y}^{valid}$
6. 比對 $\hat{y}^{valid}$ 與 $y^{valid}$ 之間的差異。

## 使用 Scikit-Learn 預測器 `LinearRegression`

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = ames_train["OverallQual"].values.reshape(-1, 1)                          # step 1
y = ames_train["SalePrice"].values                                           # step 1
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                    test_size=0.33, random_state=42) # step 2
linear_regression = LinearRegression()                                       # step 3
linear_regression.fit(X_train, y_train)                                      # step 4
y_hat = linear_regression.predict(X_valid)                                   # step 5
m = y_valid.size                                                             # step 6
mean_squared_error = ((y_valid - y_hat)**2).sum()/m                          # step 6

## 使用 Scikit-Learn 預測器 `LogisticRegression`

In [8]:
from sklearn.linear_model import LogisticRegression

csv_url = "https://raw.githubusercontent.com/datainpoint/classroom-python-for-finance-2025/refs/heads/main/titanic/train.csv"
titanic_train = pd.read_csv(csv_url)
X = titanic_train[["Parch", "Fare"]].values                                  # step 1
y = titanic_train["Survived"].values                                         # step 1
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                            test_size=0.33, random_state=42) # step 2
logistic_regression = LogisticRegression()                                   # step 3
logistic_regression.fit(X_train, y_train)                                    # step 4
y_hat = logistic_regression.predict(X_valid)                                 # step 5
number_of_misclassification = (y_valid != y_hat).sum()                       # step 6
print(number_of_misclassification)                                           # step 6

98


## 複習：Scikit-Learn 應用程式介面設計原則

- 一致性。
    - 每個轉換器類別都有 `fit_transform()` 方法。
    - 每個預測器類別都有 `fit()` 與 `predict()` 方法。
- 合理預設參數。
    - 每個轉換器、預測器都可以用預設參數建立物件。

## 複習：Scikit-Learn 應用程式介面設計原則（續）

可檢查性：每個轉換器或預測器都有屬性讓使用者檢視轉換或預測的規則。

In [9]:
print(polynomial_features.degree)
print(standard_scaler.mean_)
print(standard_scaler.scale_)
print(linear_regression.intercept_)
print(linear_regression.coef_)
print(logistic_regression.intercept_)
print(logistic_regression.coef_)

2
[6.09931507]
[1.38252284]
-86359.39607039432
[43696.47641718]
[-0.95144328]
[[0.01009638 0.01391083]]
